In [113]:
#Importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import glob
from moviepy.editor import VideoFileClip

%matplotlib inline

In [114]:
def list_images(images, cols = 2, rows = 5, cmap=None):
    """
    Display a list of images in a single figure with matplotlib.
        Parameters:
            images: List of np.arrays compatible with plt.imshow.
            cols (Default = 2): Number of columns in the figure.
            rows (Default = 5): Number of rows in the figure.
            cmap (Default = None): Used to display gray images.
    """
    plt.figure(figsize=(10, 11))
    for i, image in enumerate(images):
        plt.subplot(rows, cols, i+1)
        #Use gray scale color map if there is only one channel
        cmap = 'gray' if len(image.shape) == 2 else cmap
        plt.imshow(image, cmap = cmap)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()

In [115]:
def RGB_color_selection(image):
    """
    Apply color selection to RGB images to blackout everything except for white and yellow lane lines.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    #White color mask
    lower_threshold = np.uint8([200, 200, 200])
    upper_threshold = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(image, lower_threshold, upper_threshold)
    
    #Yellow color mask
    lower_threshold = np.uint8([175, 175,   0])
    upper_threshold = np.uint8([255, 255, 255])
    yellow_mask = cv2.inRange(image, lower_threshold, upper_threshold)
    
    #Combine white and yellow masks
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked_image = cv2.bitwise_and(image, image, mask = mask)
    
    return masked_image

In [116]:
def convert_hsv(image):
    """
    Convert RGB images to HSV.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)


In [117]:
def HSV_color_selection(image):
    """
    Apply color selection to the HSV images to blackout everything except for white and yellow lane lines.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    #Convert the input image to HSV
    converted_image = convert_hsv(image)
    
    #White color mask
    lower_threshold = np.uint8([0, 0, 210])
    upper_threshold = np.uint8([255, 30, 255])
    white_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)
    
    #Yellow color mask
    lower_threshold = np.uint8([18, 80, 80])
    upper_threshold = np.uint8([30, 255, 255])
    yellow_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)
    
    #Combine white and yellow masks
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked_image = cv2.bitwise_and(image, image, mask = mask)
    
    return masked_image

In [118]:
def convert_hsl(image):
    """
    Convert RGB images to HSL.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)



In [119]:
def HSL_color_selection(image):
    """
    Apply color selection to the HSL images to blackout everything except for white and yellow lane lines.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    #Convert the input image to HSL
    converted_image = convert_hsl(image)
    
    #White color mask
    lower_threshold = np.uint8([0, 200, 0])
    upper_threshold = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)
    
    #Yellow color mask
    lower_threshold = np.uint8([10, 0, 100])
    upper_threshold = np.uint8([40, 255, 255])
    yellow_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)
    
    #Combine white and yellow masks
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked_image = cv2.bitwise_and(image, image, mask = mask)
    
    return masked_image

In [120]:
def gray_scale(image):
    """
    Convert images to gray scale.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [121]:
def gaussian_smoothing(image, kernel_size = 13):
    """
    Apply Gaussian filter to the input image.
        Parameters:
            image: An np.array compatible with plt.imshow.
            kernel_size (Default = 13): The size of the Gaussian kernel will affect the performance of the detector.
            It must be an odd number (3, 5, 7, ...).
    """
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

In [122]:
def canny_detector(image, low_threshold = 50, high_threshold = 150):
    """
    Apply Canny Edge Detection algorithm to the input image.
        Parameters:
            image: An np.array compatible with plt.imshow.
            low_threshold (Default = 50).
            high_threshold (Default = 150).
    """
    return cv2.Canny(image, low_threshold, high_threshold)

In [123]:
def region_selection(image):
    """
    Determine and cut the region of interest in the input image.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    mask = np.zeros_like(image)   
    #Defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    #We could have used fixed numbers as the vertices of the polygon,
    #but they will not be applicable to images with different dimesnions.
    rows, cols = image.shape[:2]
    bottom_left  = [cols * 0.1, rows * 0.95]
    top_left     = [cols * 0.4, rows * 0.6]
    bottom_right = [cols * 0.9, rows * 0.95]
    top_right    = [cols * 0.6, rows * 0.6]
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [124]:
def hough_transform(image):
    """
    Determine and cut the region of interest in the input image.
        Parameters:
            image: The output of a Canny transform.
    """
    rho = 1              #Distance resolution of the accumulator in pixels.
    theta = np.pi/180    #Angle resolution of the accumulator in radians.
    threshold = 20       #Only lines that are greater than threshold will be returned.
    minLineLength = 20   #Line segments shorter than that are rejected.
    maxLineGap = 300     #Maximum allowed gap between points on the same line to link them
    return cv2.HoughLinesP(image, rho = rho, theta = theta, threshold = threshold,
                           minLineLength = minLineLength, maxLineGap = maxLineGap)

In [125]:
def draw_lines(image, lines, color = [255, 0, 0], thickness = 2):
    """
    Draw lines onto the input image.
        Parameters:
            image: An np.array compatible with plt.imshow.
            lines: The lines we want to draw.
            color (Default = red): Line color.
            thickness (Default = 2): Line thickness.
    """
    image = np.copy(image)
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(image, (x1, y1), (x2, y2), color, thickness)
    return image

In [126]:
def average_slope_intercept(lines):
    """
    Find the slope and intercept of the left and right lanes of each image.
        Parameters:
            lines: The output lines from Hough Transform.
    """
    left_lines    = [] #(slope, intercept)
    left_weights  = [] #(length,)
    right_lines   = [] #(slope, intercept)
    right_weights = [] #(length,)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            slope = (y2 - y1) / (x2 - x1)
            intercept = y1 - (slope * x1)
            length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))
            if slope < 0:
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    left_lane  = np.dot(left_weights,  left_lines) / np.sum(left_weights)  if len(left_weights) > 0 else None
    right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None
    return left_lane, right_lane

In [127]:
def pixel_points(y1, y2, line):
    """
    Converts the slope and intercept of each line into pixel points.
        Parameters:
            y1: y-value of the line's starting point.
            y2: y-value of the line's end point.
            line: The slope and intercept of the line.
    """
    if line is None:
        return None
    slope, intercept = line
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))

In [128]:
def lane_lines(image, lines):
    """
    Create full lenght lines from pixel points.
        Parameters:
            image: The input test image.
            lines: The output lines from Hough Transform.
    """
    left_lane, right_lane = average_slope_intercept(lines)
    y1 = image.shape[0]
    y2 = y1 * 0.6
    left_line  = pixel_points(y1, y2, left_lane)
    right_line = pixel_points(y1, y2, right_lane)
    return left_line, right_line

    
def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=12):
    """
    Draw lines onto the input image.
        Parameters:
            image: The input test image.
            lines: The output lines from Hough Transform.
            color (Default = red): Line color.
            thickness (Default = 12): Line thickness. 
    """
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line,  color, thickness)
    return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)
             
    


In [129]:
#Import everything needed to edit/save/watch video clips
from moviepy import *
from IPython.display import HTML
from IPython.display import Image

In [130]:
def frame_processor(image):
    """
    Process the input frame to detect lane lines.
        Parameters:
            image: Single video frame.
    """
    color_select = HSL_color_selection(image)
    gray         = gray_scale(color_select)
    smooth       = gaussian_smoothing(gray)
    edges        = canny_detector(smooth)
    region       = region_selection(edges)
    hough        = hough_transform(region)
    result       = draw_lane_lines(image, lane_lines(image, hough))
    return result 

In [131]:
def process_video(test_video, output_video):
    """
    Read input video stream and produce a video file with detected lane lines.
        Parameters:
            test_video: Input video.
            output_video: A video file with detected lane lines.
    """
    input_video = VideoFileClip(os.path.join('test_videos', test_video), audio=False)
    processed = input_video.fl_image(frame_processor)
    processed.write_videofile(os.path.join('output_videos', output_video), audio=False)

In [132]:
%time process_video('solidWhiteRight.mp4', 'solidWhiteRight_output.mp4')
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("output_videos\solidWhiteRight_output.mp4"))




                                                                                                              


t:   3%|█▊                                                                  | 18/681 [01:12<00:19, 34.87it/s, now=None]


                                                                                                              


t:   3%|█▊                                                                  | 18/681 [01:12<00:19, 34.87it/s, now=None]

Moviepy - Building video output_videos\solidWhiteRight_output.mp4.
Moviepy - Writing video output_videos\solidWhiteRight_output.mp4



                                                                                                                       


                                                                                                              


t:   3%|█▊                                                                  | 18/681 [01:20<00:19, 34.87it/s, now=None]


                                                                                                              


t:   3%|█▊                                                                  | 18/681 [01:20<00:19, 34.87it/s, now=None]

Moviepy - Done !
Moviepy - video ready output_videos\solidWhiteRight_output.mp4
CPU times: total: 6.55 s
Wall time: 8.7 s


In [133]:
%time process_video('solidYellowLeft.mp4', 'solidYellowLeft_output.mp4')
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("output_videos\solidYellowLeft_output.mp4"))




                                                                                                              


t:   3%|█▊                                                                  | 18/681 [01:20<00:19, 34.87it/s, now=None]


                                                                                                              


t:   3%|█▊                                                                  | 18/681 [01:20<00:19, 34.87it/s, now=None]

Moviepy - Building video output_videos\solidYellowLeft_output.mp4.
Moviepy - Writing video output_videos\solidYellowLeft_output.mp4



                                                                                                                       


                                                                                                              


t:   3%|█▊                                                                  | 18/681 [01:44<00:19, 34.87it/s, now=None]


                                                                                                              


t:   3%|█▊                                                                  | 18/681 [01:44<00:19, 34.87it/s, now=None]

Moviepy - Done !
Moviepy - video ready output_videos\solidYellowLeft_output.mp4
CPU times: total: 19.8 s
Wall time: 24 s


In [134]:
%time process_video('challenge.mp4', 'challenge_output.mp4')
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("output_videos\challenge_output.mp4"))




                                                                                                              


t:   3%|█▊                                                                  | 18/681 [01:44<00:19, 34.87it/s, now=None]


                                                                                                              


t:   3%|█▊                                                                  | 18/681 [01:44<00:19, 34.87it/s, now=None]

Moviepy - Building video output_videos\challenge_output.mp4.
Moviepy - Writing video output_videos\challenge_output.mp4



                                                                                                                       


                                                                                                              


t:   3%|█▊                                                                  | 18/681 [02:02<00:19, 34.87it/s, now=None]


                                                                                                              


t:   3%|█▊                                                                  | 18/681 [02:02<00:19, 34.87it/s, now=None]

Moviepy - Done !
Moviepy - video ready output_videos\challenge_output.mp4
CPU times: total: 12 s
Wall time: 18.3 s


In [139]:
%time process_video('1.mp4', '1_output.mp4')
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("output_videos\1_output.mp4"))


                                                                                                                       
t:  79%|████████████████████████████████████████████████████▋              | 195/248 [01:57<00:00, 87.61it/s, now=None]
                                                                                                                       
t:  79%|████████████████████████████████████████████████████▋              | 195/248 [01:57<00:00, 87.61it/s, now=None]

Moviepy - Building video output_videos\1_output.mp4.
Moviepy - Writing video output_videos\1_output.mp4





t:   0%|                                                                             | 0/967 [00:00<?, ?it/s, now=None]

t:   0%|▏                                                                    | 2/967 [00:00<01:25, 11.26it/s, now=None]

t:   0%|▎                                                                    | 4/967 [00:00<01:11, 13.44it/s, now=None]

t:   1%|▍                                                                    | 6/967 [00:00<01:01, 15.68it/s, now=None]

t:   1%|▌                                                                    | 8/967 [00:00<00:59, 16.12it/s, now=None]

t:   1%|▋                                                                   | 10/967 [00:00<00:57, 16.76it/s, now=None]

t:   1%|▊                                                                   | 12/967 [00:00<00:54, 17.45it/s, now=None]

t:   1%|▉                                                                   | 14/967 [00:00<00:55, 17.11it/s, now=None]

t:   2%|█▏                    

t:  16%|██████████▍                                                        | 150/967 [00:08<00:47, 17.25it/s, now=None]

t:  16%|██████████▌                                                        | 152/967 [00:08<00:46, 17.56it/s, now=None]

t:  16%|██████████▋                                                        | 154/967 [00:08<00:52, 15.39it/s, now=None]

t:  16%|██████████▊                                                        | 156/967 [00:08<00:49, 16.29it/s, now=None]

t:  16%|██████████▉                                                        | 158/967 [00:09<00:49, 16.46it/s, now=None]

t:  17%|███████████                                                        | 160/967 [00:09<00:47, 16.83it/s, now=None]

t:  17%|███████████▏                                                       | 162/967 [00:09<00:54, 14.69it/s, now=None]

t:  17%|███████████▍                                                       | 165/967 [00:09<00:48, 16.65it/s, now=None]

t:  17%|███████████▌            

t:  31%|████████████████████▋                                              | 298/967 [00:17<00:41, 16.02it/s, now=None]

t:  31%|████████████████████▊                                              | 300/967 [00:17<00:41, 16.14it/s, now=None]

t:  31%|████████████████████▉                                              | 302/967 [00:17<00:39, 16.96it/s, now=None]

t:  31%|█████████████████████                                              | 304/967 [00:17<00:38, 17.17it/s, now=None]

t:  32%|█████████████████████▏                                             | 306/967 [00:17<00:39, 16.72it/s, now=None]

t:  32%|█████████████████████▎                                             | 308/967 [00:17<00:41, 15.71it/s, now=None]

t:  32%|█████████████████████▍                                             | 310/967 [00:18<00:39, 16.44it/s, now=None]

t:  32%|█████████████████████▌                                             | 312/967 [00:18<00:43, 15.09it/s, now=None]

t:  33%|█████████████████████▊  

t:  45%|██████████████████████████████▏                                    | 435/967 [00:26<00:32, 16.19it/s, now=None]

t:  45%|██████████████████████████████▎                                    | 437/967 [00:26<00:35, 14.79it/s, now=None]

t:  45%|██████████████████████████████▍                                    | 439/967 [00:26<00:33, 15.93it/s, now=None]

t:  46%|██████████████████████████████▌                                    | 441/967 [00:26<00:32, 15.95it/s, now=None]

t:  46%|██████████████████████████████▋                                    | 443/967 [00:26<00:31, 16.70it/s, now=None]

t:  46%|██████████████████████████████▊                                    | 445/967 [00:27<00:37, 13.98it/s, now=None]

t:  46%|██████████████████████████████▉                                    | 447/967 [00:27<00:35, 14.62it/s, now=None]

t:  47%|███████████████████████████████▏                                   | 450/967 [00:27<00:30, 16.85it/s, now=None]

t:  47%|████████████████████████

t:  59%|███████████████████████████████████████▋                           | 572/967 [00:35<00:24, 16.41it/s, now=None]

t:  59%|███████████████████████████████████████▊                           | 574/967 [00:35<00:29, 13.44it/s, now=None]

t:  60%|███████████████████████████████████████▉                           | 577/967 [00:35<00:24, 15.61it/s, now=None]

t:  60%|████████████████████████████████████████                           | 579/967 [00:35<00:25, 15.07it/s, now=None]

t:  60%|████████████████████████████████████████▎                          | 581/967 [00:35<00:24, 15.80it/s, now=None]

t:  60%|████████████████████████████████████████▍                          | 583/967 [00:35<00:25, 14.99it/s, now=None]

t:  60%|████████████████████████████████████████▌                          | 585/967 [00:36<00:25, 15.25it/s, now=None]

t:  61%|████████████████████████████████████████▋                          | 587/967 [00:36<00:26, 14.26it/s, now=None]

t:  61%|████████████████████████

t:  73%|█████████████████████████████████████████████████                  | 709/967 [00:43<00:16, 15.38it/s, now=None]

t:  74%|█████████████████████████████████████████████████▎                 | 711/967 [00:44<00:15, 16.34it/s, now=None]

t:  74%|█████████████████████████████████████████████████▍                 | 713/967 [00:44<00:15, 16.18it/s, now=None]

t:  74%|█████████████████████████████████████████████████▌                 | 716/967 [00:44<00:15, 15.97it/s, now=None]

t:  74%|█████████████████████████████████████████████████▊                 | 719/967 [00:44<00:14, 17.64it/s, now=None]

t:  75%|█████████████████████████████████████████████████▉                 | 721/967 [00:44<00:14, 17.21it/s, now=None]

t:  75%|██████████████████████████████████████████████████                 | 723/967 [00:44<00:14, 17.39it/s, now=None]

t:  75%|██████████████████████████████████████████████████▏                | 725/967 [00:44<00:13, 17.33it/s, now=None]

t:  75%|████████████████████████

t:  88%|███████████████████████████████████████████████████████████        | 853/967 [00:52<00:05, 19.95it/s, now=None]

t:  89%|███████████████████████████████████████████████████████████▎       | 856/967 [00:52<00:06, 17.24it/s, now=None]

t:  89%|███████████████████████████████████████████████████████████▌       | 859/967 [00:52<00:05, 19.41it/s, now=None]

t:  89%|███████████████████████████████████████████████████████████▋       | 862/967 [00:52<00:05, 18.43it/s, now=None]

t:  89%|███████████████████████████████████████████████████████████▉       | 865/967 [00:52<00:05, 19.44it/s, now=None]

t:  90%|████████████████████████████████████████████████████████████▏      | 868/967 [00:53<00:05, 18.99it/s, now=None]

t:  90%|████████████████████████████████████████████████████████████▎      | 870/967 [00:53<00:05, 18.31it/s, now=None]

t:  90%|████████████████████████████████████████████████████████████▍      | 873/967 [00:53<00:05, 18.28it/s, now=None]

t:  90%|████████████████████████

Moviepy - Done !
Moviepy - video ready output_videos\1_output.mp4
CPU times: total: 59.6 s
Wall time: 1min


In [138]:
%time process_video('2.mp4', '2_output.mp4')
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("output_videos\2_output.mp4"))

t:  79%|████████████████████████████████████████████████████▋              | 195/248 [00:44<00:00, 87.61it/s, now=None]

Moviepy - Building video output_videos\2_output.mp4.
Moviepy - Writing video output_videos\2_output.mp4




t:  77%|███████████████████████████████████████████████████               | 192/248 [00:02<00:00, 104.27it/s, now=None]

TypeError: 'NoneType' object is not iterable

In [140]:
%time process_video('3.mp4', '3_output.mp4')
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("output_videos\3_output.mp4"))


                                                                                                                       
t:  79%|████████████████████████████████████████████████████▋              | 195/248 [06:31<00:00, 87.61it/s, now=None]
                                                                                                                       
t:  79%|████████████████████████████████████████████████████▋              | 195/248 [06:31<00:00, 87.61it/s, now=None]

Moviepy - Building video output_videos\3_output.mp4.
Moviepy - Writing video output_videos\3_output.mp4





t:   0%|                                                                            | 0/1730 [00:00<?, ?it/s, now=None]

t:   0%|                                                                    | 2/1730 [00:00<02:11, 13.15it/s, now=None]

OverflowError: cannot convert float infinity to integer